# Uncheck if you want to install PyTMD

In [ ]:
#pip install pyTMD

In [ ]:
import os, sys
from datetime import datetime, date, time, timezone, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

#Importing the PyTMD functions. The PyTMD libraries have already been installed.
import pyTMD.io
import pyTMD.time
import pyTMD.predict
import pyTMD.tools
import pyTMD.utilities

# autoreload
%load_ext autoreload
%autoreload 2

%load_ext autoreload

pwd = os.getcwd()

In [ ]:
# First changing the working directory to current as all the paths are set to current working directory
os.chdir(pwd)


# update the tide prediction and plot
def update_tide_prediction():
    model = pyTMD.io.model('folder where the model is saved', format='FES',
                compressed=False).elevation('FES2014')
    #folder = C:\\Users\\...\\OneDrive - University of New Brunswick\\Fall 2022
    #format = netcdf/FES/OTIS
    #elevation = TPXO9-atlas-v5/FES2014/GOT4.10
    
    #Open the .txt file containing the datetime and location (Longitude and latitude) values
    df1 = pd.read_csv("DataFolder")
    
    #Sort the data by timestamp, just incase they are scattered.
    df = df1.sort_values(by='datetime')
    
    
    #Calling the coordinates of the survey track
    LAT = df['latitude']
    LON = df['longitude']    
    
    # convert date-time to timestamp
    dates = df['datetime']
    d = list(map(lambda i: np.datetime64(datetime.strptime(dates[i], '%Y-%m-%d %H:%M:%S.%f')), range(len(dates))))
    
    #converting the timestamps to Modified Julian Date (MJD
    tide_time = np.array(pyTMD.time.convert_datetime(d, ep)och =pyTMD.time._tide_epoch)/86400)  

    # number of time points
    nt = len(tide_time)

    # delta time (TT - UT1) file
    delta_file = pyTMD.utilities.get_data_path(['data','merged_deltat.data'])
    

    #Informs the code to predict tide using the FES2014 parameters
    if model.format == 'FES':
        constituents = pyTMD.io.FES.read_constants(model.model_file,
            type=model.type, version=model.version,
            compressed=model.compressed)
        c = model.constituents
        # interpolate delta times from calendar dates to tide time
        DELTAT = pyTMD.time.interpolate_delta_time(delta_file, tide_time)
        
        amp,ph = pyTMD.io.FES.interpolate_constants(
            np.atleast_1d(LON), np.atleast_1d(LAT),
            constituents, scale=model.scale,
            method='spline', extrapolate=True)
        
    # calculate complex phase in radians for Euler's
    cph = -1j*ph*np.pi/180.0
    # calculate constituent oscillation
    hc = amp*np.exp(cph)

    # predict tidal elevations at time 1 and infer minor corrections
    TIDE = pyTMD.predict.drift(tide_time, hc, c,
        deltat=DELTAT, corrections=model.format)
    MINOR = pyTMD.predict.infer_minor(tide_time, hc, c,
        deltat=DELTAT, corrections=model.format)
    TIDE.data[:] += MINOR.data[:]
    # convert to meters
    TIDE.data[:] *= 1
    
    #Defining the header in DataFrame (FES2014 values are measured to Mean Sea Surface[MSS])
    df['FES_MSS'] = TIDE

    
    # Save the updated DataFrame to a new CSV file if needed
    df.to_csv('updated_Folder.txt', sep=',', index=False)
    
    
# run tide prediction at initial location
update_tide_prediction()